# Eploratory Analysis Phishing_Email.csv Group 3

In [42]:
# Potentionally helpful libraries:
# pandas, matplotlib, seaborn, nltk, scikit-learn, gensim, pyLDAvis

import pandas as pd
import re

In [44]:
# Reading in the data
df_phish = pd.read_csv("Phishing_Email.csv")
# Dropping the unecessary index row
df_phish = df_phish.drop(df_phish.columns[0], axis=1)
#checking that it read in correctly
df_phish[0:10]

,Email Text,Email Type
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,software at incredibly low prices ( 86 % lower...,Phishing Email
5,global risk management operations sally congra...,Safe Email
6,"On Sun, Aug 11, 2002 at 11:17:47AM +0100, wint...",Safe Email
7,"entourage , stockmogul newsletter ralph velez ...",Phishing Email
8,"we owe you lots of money dear applicant , afte...",Phishing Email
9,re : coastal deal - with exxon participation u...,Safe Email


In [61]:
# Pulling out a few Emails as examples
for i in range(2):
    print("\n\nBEGIN NEW EMAIL:")
    print(df_phish.loc[i+22,"Email Text"])






BEGIN NEW EMAIL:

Question?Do you want a different job?
Do you want to be your own boss?
Do you need extra income?
Do you need to start a new life?
Does your current job seem to go nowhere?If you answered yes to these questions, then here is your solution.We are a fortune 500 company looking for motivated individuals who are 
looking
to a substantial income working from home.Thousands of individual are currently do this RIGHT NOW.
So if you are looking to be employed at home, with a career that will
provide you vast opportunities and a substantial income, please fill
out our online information request form here now:http://ter.netblah.com:27000To miss out on this opportunity, click herehttp://ter.netblah.com:27000/remove.html



BEGIN NEW EMAIL:
URL: http://www.livejournal.com/talkread.bml?journal=jwz&itemid=63309
Date: Not suppliedhttp://www.livejournal.com/talkread.bml?journal=jwz&itemid=63309



## Feature Extraction

### Emails with "re:"

In [ ]:
# Adding "Is_Response" feature
df_phish["Is_Response"] = None
# Looping over the DataFrame
for i in range(df_phish.shape[0]):
    hasRE = re.search('''re :''', str(df_phish.iloc[i]["Email Text"]))
    if hasRE:
        df_phish.at[i, "Is_Response"] = True
    else:
        df_phish.at[i, "Is_Response"] = False

In [56]:
df_phish["Is_Response"].value_counts(normalize=True)

Is_Response
False    0.870563
True     0.129437
Name: proportion, dtype: float64

### Emails with links

In [51]:
# Adding "Has_HyperLink" feature column
df_phish["Has_WebLink"] = None
# Patterns to search for hyperlinks, This may need updating to include more patterns
patterns = [r'https?://\S+', r'www\.\S+']

# Looping over the DataFrame
for i in range(df_phish.shape[0]):
    has_link = False
    # Looping over each pattern
    for pattern in patterns:
        if re.search(pattern, str(df_phish.iloc[i]["Email Text"])):
            has_link = True
            break  # Break the loop if any pattern is found
    # Update the "Has_WebLink" column based on whether a link is found
    df_phish.at[i, "Has_WebLink"] = has_link

In [55]:
df_phish["Has_WebLink"].value_counts(normalize=True)

Has_WebLink
False    0.775979
True     0.224021
Name: proportion, dtype: float64